# Свойства многофазного потока по uniflocvba и uniflocpy

In [ ]:
import sys
sys.path.append('../')

import uniflocpy.uTools.data_workflow as data_workflow
import pandas as pd
from uniflocpy.uTools import plotly_workflow
import uniflocpy.uPVT.BlackOil_model as BlackOil_model
import uniflocpy.uValidation.python_api as python_api
import uniflocpy.uTools.uconst as uc
import uniflocpy.uPVT.PVT_fluids as PVT_fluids
import numpy as np

In [ ]:
def create_result_df_from_vba_output(vba_result, p_bar, t_c):
    result_dict = {}
    for key, value in zip(vba_result[1],vba_result[0]):
        result_dict[key] = value
    result_dict['p_bar'] = p_bar
    result_dict['t_c'] = t_c
    this_df = pd.DataFrame(result_dict, index = [0])
    return this_df

Загрузка надстройки

In [ ]:
uniflocvba = python_api.API('E:\\Git\\unifloc_vba\\UniflocVBA_7.xlam')

Задание параметров

In [ ]:
gamma_oil =0.86
gamma_water = 1
gamma_gas = 1.45  * 24.05 / 28.98
rsb_m3m3 = 56
t_res_c = 40
t_c = 30
p_bar = 90
q_liq_sm3day = 40
watercut_perc = 0


keywords_python = {"gamma_oil": gamma_oil, "gamma_gas": gamma_gas, "gamma_wat":gamma_water,
                                    "rsb_m3m3": rsb_m3m3, "t_res_c": t_res_c, 'pb_bar':90}

keywords_vba = {"qliq_sm3day":q_liq_sm3day, "fw_perc":watercut_perc,
                "t_C": t_c, "gamma_gas": gamma_gas,
                "gamma_oil": gamma_oil, "gamma_wat": gamma_water, "rsb_m3m3": rsb_m3m3, "tres_C": t_res_c}



In [ ]:
15*10**(-6)/0.0503

Настройка модели флюида

In [ ]:
blackoil_option = BlackOil_model.BlackOil_option()
blackoil_option.b_wat_cor_number = 1
blackoil_option.mu_wat_cor_number = 1
blackoil_option.rho_wat_cor_number = 1
blackoil_option.z_cor_number = 1
blackoil_option.pseudocritical_temperature_cor_number = 1
blackoil_option.pseudocritical_pressure_cor_number = 1
blackoil_option.rho_gas_cor_number = 1
blackoil_option.b_gas_cor_number = 1 - 1
blackoil_option.mu_dead_oil_cor_number = 2
blackoil_option.sigma_oil_gas_cor_number = 2
blackoil_option.sigma_wat_gas_cor_number = 1

#blackoil_option.rs_cor_number =2
blackoil_option.pb_cor_number = 3

blackoil_option.set_vba_preset()


python_fluid = BlackOil_model.Fluid(**keywords_python, option = blackoil_option)

Инициализация многофазного потока

In [ ]:
multiphase_flow = PVT_fluids.FluidFlow(python_fluid)
multiphase_flow.qliq_on_surface_m3day = q_liq_sm3day
multiphase_flow.fw_on_surface_perc  =watercut_perc

In [ ]:
rus_fluid_flow = BlackOil_model.Fluid(**keywords_python, activate_rus_cor=1, p_reservoir_bar=100)

In [ ]:
multiphase_flow2 = PVT_fluids.FluidFlow(rus_fluid_flow)
multiphase_flow2.qliq_on_surface_m3day = q_liq_sm3day
multiphase_flow2.fw_on_surface_perc  =watercut_perc

In [ ]:
rus_fluid_flow = multiphase_flow2

In [ ]:
def calc_t(p_y, p_bhp, t_y, t_bhp, p):
    return t_y + ((t_bhp - t_y)*(p-p_y)/(p_bhp-p_y))

Расчет

In [ ]:
python_flow_model_db = data_workflow.Data()
python_flow_model_db.clear_data()



rus_flow_model_db = data_workflow.Data()
rus_flow_model_db.clear_data()

vba_result_df = None
for p_atm in range(1, 145, 1):
    
    p_bar = uc.bar2atm(p_atm+1)
    t_k = calc_t(9, 114.35, 295, 313,p_bar)
    t_c = uc.k2c(t_k)
    multiphase_flow.calc(p_bar, t_c)
    p_atm = uc.bar2atm(p_bar)
    
    rus_fluid_flow.calc(p_bar, t_c)
    rus_flow_model_db.get_data(rus_fluid_flow, object_name='rus_flow_model')
        
    python_flow_model_db.get_data(multiphase_flow, object_name='python_flow_model')
    vba_result = uniflocvba.MF_all_mf(p_atma = p_atm, **keywords_vba)
    
    
    this_vba_result_df = create_result_df_from_vba_output(vba_result, p_bar, t_c)
    try:
        vba_result_df = vba_result_df.append(this_vba_result_df)
    except:
        vba_result_df = this_vba_result_df.copy()

Объединение результатов

In [ ]:
result_unifloc_python = python_flow_model_db.get_data_as_df()
result_unifloc_python.index = result_unifloc_python['python_flow_model.p_bar']
result_unifloc_python.index.name = 'p_bar'


result_ru = rus_flow_model_db.get_data_as_df()
result_ru.index = result_ru['rus_flow_model.p_bar']
result_ru.index.name = 'p_bar'
result_ru = result_ru.add_prefix('ru_')

vba_result_df.index = result_unifloc_python.index
vba_result_df.index.name = 'p_bar'
vba_result_df['mn_kgsec'] = vba_result_df['mw_kgsec'] + vba_result_df['mg_kgsec'] + vba_result_df['mo_kgsec']
vba_result_df['liquid_content_d'] = 1 - vba_result_df['gas_fraction_d']
vba_result_df = vba_result_df.add_prefix('vba_flow_model.')
all_result = result_unifloc_python.join(vba_result_df)
all_result = all_result.join(result_ru)

In [ ]:
all_result = result_ru
all_result.index = uc.bar2atm(all_result.index) - np.min(uc.bar2atm(all_result.index))

Построение графиков

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [#['dissolved_gas_sc_m3m3'],
                                              #['liberated_gas_sc_m3m3'],
                                              #['m3m3m'],
                                               ['gas_liberated_m3m3'],
                                              ['ru', 'rs_m3m3']], ['cor_number', 'cal', 'max'], swap_xy=False, reversed_y=False, 
                                 plot_name='Объем растворенного и выделившегося газа', 
                                x_name = 'Давление избыточное, атм', 
                                y_name = 'Объем газа, м3/м3',
                                 traces_names = ['Объем выделившегося  газа по Мищенко, м3/м3', 
                                                'Объем растворенного газа по Мищенко, м3/м3']
                                                #'Объем выделившегося газа по Мищенко, м3/м3',
                                                #'Объем растворенного газа по Мищенко, м3/м3' ]
                                                
                                 )

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['gas_fraction'], ['liquid_content'],
                                              ], ['vba','fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  
                                plot_name='Объемная расходная доля жидкости и газа в потоке ГЖС (без учета проскальзывания)', 
                                x_name = 'Давление избыточное, атм', 
                                y_name = 'д.ед.',
                                 traces_names = ['Объемная расходное газосодержание в потоке ГЖС, д.ед.', 
                                                'Объемная доля жидкости в потоке ГЖС, д.ед.' 
                                                ])

In [ ]:
plotly_workflow.plot_by_patterns(all_result, ['pb_bar'], ['cor_number'], swap_xy=False, reversed_y=False,  
                                 plot_name='Давление насыщения', 
                                x_name = 'Давление избыточное, атм',
                                y_name = 'Давление насыщения, бар',
                                traces_names = ['Давление насыщения по Стендингу, бар', 
                                                'Давление насыщения по исходным данным, бар'])

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['mu_gas_cp']], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,
                                 plot_name='Вязкость газа', 
                                 x_name = 'Давление избыточное, атм',
                                y_name = 'Вязкость газа, сПуаз',
                                #traces_names = ['Давление насыщения по Стендингу, бар', 
                                #                'Вязкость нефти, сПуаз']
                        )

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['z']], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,
                                 plot_name='Коэффициент сверхсжимаемости', 
                                 x_name = 'Давление избыточное, атм',
                                y_name = 'Коэффициент сверхсжимаемости',
                                traces_names = ['Давление насыщения по Стендингу, бар', 
                                                'Вязкость нефти, сПуаз']
                        )

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['b_oil']], ['cor_number', 'cal', 'stkgm', 'b_oil_b'], swap_xy=False, reversed_y=False,
                                 plot_name='Объемный коэффициент нефти', 
                                 x_name = 'Давление избыточное, атм',
                                y_name = 'Объемный коэффициент нефти, м3/м3',
                                #traces_names = ['Давление насыщения по Стендингу, бар', 
                                #                'Вязкость нефти, сПуаз']
                                )

In [ ]:
plotly_workflow.plot_by_patterns(all_result, [['flow_model.mu_liq'],['mun'],['fl.mu']], ['dead','vba','oil_b','cor_number'], swap_xy=False, reversed_y=False, 
                                plot_name='Вязкость ГЖС и составляющих', 
                                x_name = 'Давление избыточное, атм', 
                                y_name = 'Вязкость, сПуаз',
                                 traces_names = ['Вязкость жидкости, сПуаз', 
                                                'Объемная вязкость ГЖС, сПуаз', 
                                                'Вязкость нефти, сПуаз',
                                                'Вязкость газа, сПуаз',
                                                'Вязкость воды, сПуаз']
                                 )

In [ ]:
plotly_workflow.plot_by_patterns(all_result, [['flow_model.rho'],['fl.rho']], ['above','_d','dead','vba','oil_b','cor_number', 'stkgm'], swap_xy=False, reversed_y=False, 
                                plot_name='Расходные плотности без учета проскальзывания при данных термобарических условиях', 
                                x_name = 'Давление избыточное, атм', 
                                y_name = 'Плотность, кг/м3',
                                traces_names = ['Расходная плотность жидкости, кг/м3', 
                                               'Расходная плотность ГЖС, кг/м3', 
                                               'Плотность нефти, кг/м3',
                                               'Плотность газа, кг/м3',
                                               'Плотность воды, кг/м3']
                                
                                 
                                )

In [ ]:
plotly_workflow.plot_by_patterns(all_result, ['rho'], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [ ["m3t"]], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["heatcap"], ["cliq_JkgC"]], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["sigma"]], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["kgsec"]], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

# Суммарный массовый расход смеси должен быть постоянным

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["q", 'oil_m3day']], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False, 
                             plot_name = 'Дебит нефти при данных термобарических условиях',
                                 x_name = 'Давление избыточное, атм', 
                                y_name = 'Дебит нефти при данных P,T, м3/сут',
                                traces_names = ['Расходная плотность жидкости, кг/м3', 
                                               'Расходная плотность ГЖС, кг/м3', 
                                               'Плотность нефти, кг/м3',
                                               'Плотность газа, кг/м3',
                                               'Плотность воды, кг/м3'])

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["q", 'liq']], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["q", 'wat']], ['fl.','cor_number', 'mass_fraction', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["q", 'gas_m3day']], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False, 
                                plot_name = 'Дебит газа при данных термобарических условиях',
                                 x_name = 'Давление избыточное, атм', 
                                y_name = 'Дебит газа при данных P,T, м3/сут',
                                traces_names = ['Расходная плотность жидкости, кг/м3', 
                                               'Расходная плотность ГЖС, кг/м3', 
                                               'Плотность нефти, кг/м3',
                                               'Плотность газа, кг/м3',
                                               'Плотность воды, кг/м3']
                                )

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['rs'], ['liberated_gas'], ['dissolved_gas_sc_m3m3']], ['cor_number', 'cal', 'rsw'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['liq', 'm3day']], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['mix', 'm3day']], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  ['fw'], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['liquid_content']], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['rs']], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['rho_oil_kgm3']], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,
                                plot_name='Плотность газонасыщенной нефти',
                                 x_name = 'Давление избыточное, бар', 
                                y_name = 'Плотность нефти, кг/м3',
                                traces_names = [
                                               'Вязкость, сПуаз',
                                               ])

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['b_oil_m3m3']], ['cor_number', 'cal'], swap_xy=False, reversed_y=False, 
                                 plot_name='Объемный коэффициент газонасыщенной нефти',
                                 x_name = 'Давление, бар', 
                                y_name = 'bн, м3/м3',
                                traces_names = [
                                               'Вязкость, сПуаз',
                                               ]
                                )

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['mu_oil_cp']], ['cor_number', 'cal'], 
                                 swap_xy=False, reversed_y=False,  plot_name='Вязкость газонасыщенной нефти',
                                x_name = 'Давление, бар', 
                                y_name = 'Вязкость, сПуаз',
                                traces_names = [
                                               'Вязкость нефти, сПуаз',
                                               ])